# 準備

## import

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import re
import pandas as pd

## データの読み込み

### 削除するカラム

In [ ]:
drop_list = [
    "tv",
    "referee",
    "time",
    "home",
    "away",
    "home_01", "home_02", "home_03", "home_04", "home_05", "home_06", "home_07", "home_08", "home_09", "home_10", "home_11",
    "away_01", "away_02", "away_03", "away_04", "away_05", "away_06", "away_07", "away_08", "away_09", "away_10", "away_11",
    "address",
    "humidity",
]

### データ

In [ ]:
datas = pd.read_csv("./input/train_all.csv")


datas = datas.drop(drop_list, axis=1)

datas = datas.sort_values(by="y")

datas.head(5)


## 外れ値の除外

In [ ]:
datas = datas[datas["id"] != 15127]	# yの値が大きく外れたデータ
datas = datas[datas["id"] != 15699]	# yの値が0のデータ
datas = datas[datas["id"] != 14071]	# 木の中でyの値が外れたデータ
datas = datas[datas["id"] != 14911]	# 金の中でyの値が外れたデータ

## 前処理

### 日付関係

In [ ]:
datas["MONTH"] = datas["gameday"].apply(lambda x : x[0:2])
datas["DAY"] = datas["gameday"].apply(lambda x : x[3:5])
datas["WEEK"] = datas["gameday"].apply(lambda x : x[x.find("(")+1:x.find("(")+2])
datas = pd.get_dummies(datas, columns=["WEEK"], drop_first=True)
datas = datas.drop("gameday", axis=1)

### 試合？関係

In [ ]:
datas["stage"] = datas["stage"].apply(lambda x : x[1])
datas["match_num"] = datas["match"].apply(lambda x : int(re.findall(r'\d+', x)[0]))
datas["match_day"] = datas["match"].apply(lambda x : int(re.findall(r'\d+', x)[1]))
datas = datas.drop("match", axis=1)

### チーム関係

In [ ]:
datas = pd.get_dummies(datas, columns=["home_team"], drop_first=True)
datas = pd.get_dummies(datas, columns=["away_team"], drop_first=True)

### スタジアム関係

In [ ]:
datas = pd.get_dummies(datas, columns=["stadium"], drop_first=True)


## 説明変数

In [ ]:
# 説明変数を設定
not_need_cols = [
	"id",
	"weather",
	"y",
]

In [ ]:
X = datas.drop(not_need_cols, axis=1)
y = datas["y"]

In [ ]:
X.head()

# 訓練・テスト

## テスト用データにのみ教師データがあるのでテスト用データを訓練データとテストデータに分ける

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## 学習とスコアの表示

### Ridge回帰

In [ ]:
ridge100 = Ridge(alpha=100).fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge100.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge100.score(X_test, y_test)))

In [ ]:
ridge10 = Ridge(alpha=10).fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge10.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge10.score(X_test, y_test)))

In [ ]:
ridge1 = Ridge().fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge1.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge1.score(X_test, y_test)))

In [ ]:
ridge01 = Ridge(alpha=0.1).fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge01.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge01.score(X_test, y_test)))

In [ ]:
ridge001 = Ridge(alpha=0.01).fit(X_train, y_train)
print("Training set score: {:.2f}".format(ridge001.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge001.score(X_test, y_test)))

### 通常最小二乗法

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))

### Lasso

In [ ]:
lasso1 = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso1.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso1.score(X_test, y_test)))

In [ ]:
lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso1.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso1.score(X_test, y_test)))

In [ ]:
lasso00001 = Lasso(alpha=0.0001, max_iter=100000).fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso1.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso1.score(X_test, y_test)))